In [2]:
# carregando a base de dados
from keras.datasets import cifar10
(trainX, trainY), (testX, testY) = cifar10.load_data()
print(trainX.shape)
print(trainY.shape)
print(testX.shape)
print(testY.shape)

(50000, 32, 32, 3)
(50000, 1)
(10000, 32, 32, 3)
(10000, 1)


In [3]:
# Alterando tamanho das imagens
import cv2
import numpy as np
w=64
h=64
Xtrain_new=[]
#ytrain_new=[]

# aumentando o tamanho das imagens de treinamento
for i in range(trainX.shape[0]):
  im=cv2.resize(trainX[i], (w, h))
  Xtrain_new.append(im)
#  ytrain_new.append(trainY[i])

# aumentando o tamanho das imagens de teste
Xtest_new=[]
for i in range(testX.shape[0]):
  im=cv2.resize(testX[i], (w, h))
  Xtest_new.append(im)

# transformando lista em array
trainX=np.asarray(Xtrain_new)
testX=np.asarray(Xtest_new)
#trainY=np.asarray(ytrain_new)

del(Xtrain_new)
del(Xtest_new)
#del(ytrain_new)

print(trainX.shape)
print(testX.shape)

(50000, 64, 64, 3)
(10000, 64, 64, 3)


In [4]:
# Preparando os arquivos de rótulos
from tensorflow.keras.utils import to_categorical
trainY = to_categorical(trainY)
testY = to_categorical(testY)

print(trainY.shape)

(50000, 10)


In [5]:
# Importando a CNN pré-treinada para o ajuste fino

from keras.models import Sequential
from keras.callbacks import EarlyStopping
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
from keras import regularizers
import keras
import tensorflow as tf
import os
import random
import numpy as np

# SEEDS PARA REPRODUZIR RESULTADOS - QUALQUER NUMERO
RANDOM_SEED = 42

# tensorflow
tf.random.set_seed(RANDOM_SEED)

# numpy
np.random.seed(RANDOM_SEED)

# Definindo nossa CNN
model = Sequential()

# Parte convolucional
model.add(Conv2D(32, (5,5), padding='same', activation='relu', kernel_regularizer=regularizers.l2(1e-4), input_shape=(w, h, 3)))
model.add(Conv2D(32, (5,5), padding='same', activation='relu', kernel_regularizer=regularizers.l2(1e-4)))
#model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(64, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l2(1e-4)))
model.add(Conv2D(64, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l2(1e-4)))
#model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(128, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l2(1e-4)))
model.add(Conv2D(128, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l2(1e-4)))
#model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

# Parte totalmente conectada (densa)
model.add(Flatten())
#model.add(GlobalAveragePooling2D())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(10, activation='softmax'))

# Compile the model
model.compile(loss=keras.losses.categorical_crossentropy, optimizer='adam', metrics=['accuracy'])
model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 64, 64, 32)        2432      
                                                                 
 conv2d_1 (Conv2D)           (None, 64, 64, 32)        25632     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 32, 32, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 32, 32, 32)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 32, 32, 64)        18496     
                                                                 
 conv2d_3 (Conv2D)           (None, 32, 32, 64)        36928     
                                                      

In [ ]:
# Traning the model
#es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)

!mkdir checkpoints
fp='/content/checkpoints/bestmodel.hdf5'
mc = tf.keras.callbacks.ModelCheckpoint(
    filepath=fp,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True,
    verbose=1)

history = model.fit(trainX, trainY, 
          batch_size=128,
          epochs=50,
          verbose=1,
          validation_split=0.2,
          callbacks=[mc])

# evaluate model
_, acc = model.evaluate(testX, testY, verbose=0)
print('Final Accuracy: > %.3f' % (acc * 100.0))

Epoch 1/50
 70/313 [=====>........................] - ETA: 22:36 - loss: 2.8431 - accuracy: 0.1235

KeyboardInterrupt: ignored